<a href="https://colab.research.google.com/github/anikaSeshan/LandfillNetResearch/blob/main/DataProcessingNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import io

In [ ]:
uploaded = files.upload()

Saving lmopcompositedataProject.xlsx to lmopcompositedataProject (1).xlsx


In [ ]:
xlsx_file = io.BytesIO(uploaded.get('lmopcompositedataProject (1).xlsx'))
df = pd.read_excel(xlsx_file, "LMOP Filtered Data", usecols="B,C,E,F,J,K,L,W")
noOneStatesDf = df[df["State"] != 'AK']
noTwoStatesDf = noOneStatesDf[noOneStatesDf["State"] != 'HI'] # Filter out military landfill
completeFilteredDf = noTwoStatesDf[noTwoStatesDf["Military Landfill?"] != 'Yes']
completeFilteredDf = completeFilteredDf[completeFilteredDf["Landfill Name"] != 'NW Oklahoma SWDA LF']
completeFilteredDf = completeFilteredDf[completeFilteredDf["Landfill Name"] != 'Butte County Landfill']
completeFilteredDf = completeFilteredDf[completeFilteredDf["Landfill Name"] != 'Glenns Ferry Landfill']
completeFilteredDf = completeFilteredDf[completeFilteredDf["Landfill Name"] != 'Somerset County Municipal Landfill']
completeFilteredDf = completeFilteredDf[completeFilteredDf["Landfill Name"] != 'Earth Complex']
completeFilteredDf = completeFilteredDf[completeFilteredDf["Landfill Name"] != 'Eskdale LF']
completeFilteredDf = completeFilteredDf[completeFilteredDf["Landfill Name"] != 'Partoun LF']
completeFilteredDf = completeFilteredDf[completeFilteredDf["State"] != 'PR']
completeFilteredDf.head(15)

,Landfill ID,Landfill Name,State,Physical Address,Latitude,Longitude,Military Landfill?,Current Landfill Status
5,10860,Arrowhead Landfill,AL,On Cahaba Road\n622 Tayloe Road,32.413000,-87.468500,No,Open
6,2005,Black Warrior Solid Waste Facility,AL,3301 Landfill Drive,33.244043,-87.652095,No,Open
7,2408,Brundidge Landfill,AL,515 Cleanwater Drive,31.704100,-85.829900,No,Open
8,8,Chastang SLF,AL,17045 Highway 43,31.061790,-88.025030,No,Open
9,2,City of Decatur-Morgan County Regional MSW Lan...,AL,500 Landfill Drive,34.617500,-87.101630,No,Open
10,19,City of Dothan SLF MSWLF,AL,1290 Burkett Road,31.238766,-85.356721,No,Open
11,9,Coffee County Sanitary Landfill,AL,7733 Highway 125,31.507053,-86.000975,No,Open
12,4,Cullman Environmental Waste Management Center,AL,2805 AL Highway 69 South,34.028780,-86.900690,No,Open
13,15,Eastern Area Landfill,AL,2787 Alton Drive,33.597800,-86.628800,No,Open
14,11,Huntsville MSWLF,AL,4100 Leeman Ferry Road SW,34.688500,-86.592000,No,Open


In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project = 'ee-my-amazing-username')

In [ ]:
from IPython.display import Image, display
import geopy
import geopy.distance
import math
import folium

def get_rectangle_bounds(coordinates, width, length):
    start = geopy.Point(coordinates)
    hypotenuse = math.hypot(width/1000, length/1000)

    # Edit used wrong formula to convert radians to degrees, use math builtin function
    northeast_angle = 0 - math.degrees(math.atan(width/length))
    southwest_angle = 180 - math.degrees(math.atan(width/length))

    d = geopy.distance.distance(kilometers=hypotenuse/2)
    northeast = d.destination(point=start, bearing=northeast_angle)
    southwest = d.destination(point=start, bearing=southwest_angle)
    bounds = []
    for point in [northeast, southwest]:
        coords = (point.latitude, point.longitude)
        bounds.append(coords)

    return bounds

# To get a rotated rectangle at a bearing, you need to get the points of the the recatangle at that bearing
def get_rotated_points(coordinates, bearing, width, length):
    start = geopy.Point(coordinates)
    width  = width/1000
    length = length/1000
    rectlength = geopy.distance.distance(kilometers=length)
    rectwidth = geopy.distance.distance(kilometers=width)
    halfwidth = geopy.distance.distance(kilometers=width/2)
    halflength = geopy.distance.distance(kilometers=length/2)

    pointAB = halflength.destination(point=start, bearing=bearing)
    pointA = halfwidth.destination(point=pointAB, bearing=0-bearing)
    pointB = rectwidth.destination(point=pointA, bearing=180-bearing)
    pointC = rectlength.destination(point=pointB, bearing=bearing-180)
    pointD = rectwidth.destination(point=pointC, bearing=0-bearing)

    points = []
    for point in [pointA, pointB, pointC, pointD]:
        coords = (point.latitude, point.longitude)
        points.append(coords)

    return points


def get_image_with_coordinates(latitude, longitude, landfill_id):


  start_coords = [latitude,	longitude] # Figure out how to insert lat/long from pandas into here - create a loop (ISSUE: EXLUDING HAWAII AND ALASKA CHANGES INDEX - 314)
  length = 1000.50 #in meters                Many locations do not have center coordinates - does not look like landfill
  width = 940.50
  bearing = 45 #degrees

  m = folium.Map(start_coords, zoom_start=20)
  bounds = get_rectangle_bounds(tuple(start_coords),width, length )
  points = get_rotated_points(tuple(start_coords), bearing, width, length)

  # Create dataset

  dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2020-01-01', '2022-12-31'))
  image = dataset.select(['R', 'G', 'B']).mean()

  region = ee.Geometry.Rectangle([bounds[0][1], bounds[0][0], bounds[1][1], bounds[1][0]])
  url = image.getThumbUrl({'min': 0, 'max': 255, 'dimensions': 512, 'region': region})
  '''
  print(url)
  img = Image(url=url)
  display(img)
  '''

  task = ee.batch.Export.image.toDrive(**{
      'image': image,
      'description': str(landfill_id),
      'folder': "AI Project Dataset Negative Images Final",
      'scale': 1,
      'region': region
  })

  task.start()



In [ ]:
from IPython.display import Image, display


dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2020-01-01', '2022-12-31'))
image = dataset.select(['R', 'G', 'B']).mean()

region = ee.Geometry.Rectangle([-87.47530208206177,32.41119063441768, -87.46283517456055, 32.42068242965588])
url = image.getThumbUrl({'min': 0, 'max': 255, 'dimensions': 512, 'region': region})

print(url)
img = Image(url=url)
display(img)



https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/5886987e45e8219198e3c959c79c46ea-00f3f9e5c97f9cee5286577a5ab4fe96:getPixels


In [ ]:
import random


one_percent = int(0.01 * len(completeFilteredDf))

random_landfills = completeFilteredDf.sample(n=one_percent)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: check if '/content/drive/MyDrive/AI Project Dataset Positive Images/2048.tif exists and then delete it
import os

delete = [ 8
,4
,13
,20
,5
,11550
,11544
,2052
,11541
,2291
,11421
,11545
,11549
,1773
,12210
,1787
,2390
,2411
,1771
,1789
,20695
,40
,45
,74
,108
,139
,153
,265
,192
,200
,206
,250
,258
,286
,287
,360
,1832
,1831
,20698
,12213
,2278
,419
,420
,404
,2267
,10405
,441
,10441
,11500
,10443
,454
,448
,20699
,461
,20757
,619
,11601
,21166
,634
,622
,2181
,1864
,2306
,471
,2309
,2315
,12144
,1873
,2311
,532
,538
,642
,635
,675
,674
,688
,693
,700
,787
,734
,10740
,10720
,2270
,754
,755
,764
,2250
,2232
,2271
,1890
,2236
,2237
,2239
,2240
,1940
,890
,21135
,844
,1973
,1964
,1967
,21014
,21025
,2424
,2142
,1071
,1098
,1065
,1105
,10781
,2064
,1754
,1748
,1755
,2020
,941
,949
,20762
,21090
,1013
,992
,1001
,1202
,1908
,2434
,20714
,1243
,1295
,1286
,10600
,2202
,1415
,1461
,2439
,2273
,1530
,10080
,1575
,2133
,1606
,20759
,11040
,1657
,2220
,2210]

for i in delete:
  if os.path.exists(f'/content/drive/MyDrive/AI Project Dataset Positive Images/{i}.tif'):
    print('File exists, deleting')
    os.remove(f'/content/drive/MyDrive/AI Project Dataset Positive Images/{i}.tif')
  else:
    print('File does not exist')


File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist
File does not exist


In [ ]:
for index, row in completeFilteredDf.iterrows():
    print(row['State'], row['Landfill Name'], row['Landfill ID'])
    get_image_with_coordinates(row['Latitude'], row['Longitude'], row['Landfill ID'])

AL Arrowhead Landfill 10860
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/6701483ff7cab4f792681dd53d14dc2f-b87b14a5ebe7f260332153c09214fce6:getPixels


AL Black Warrior Solid Waste Facility 2005
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/ff63d06c4875745ccf929b24c3a10734-7280898caa587cde67be079980887c21:getPixels


AL Brundidge Landfill 2408
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/cef2336a927123f4d9ecb1c73ed4e4eb-81d58ac6eb6c4612348b7240b90aa150:getPixels


AL Chastang SLF 8
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/1522eaea6023296ee979c9152aac679a-068987aa2e8820e03954b34a9d49bde3:getPixels


AL City of Decatur-Morgan County Regional MSW Landfill 2
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/ad822f977d7b77fd7344ea194106c5b5-df9b2053ceee27d99e2aeb7df296a26d:getPixels


...

WY Campbell County Landfill No.2 20722
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/7ee42dc7743a0df99c4a319bba203e8f-0ad0f9a4385b74a7df441c1cfd7d6d5a:getPixels


WY Casper Balefill 11100
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/e9a7b6097dcc416126183af9d10fa150-29998b0785c2736545a0da74bcb9a0ef:getPixels


WY Cheyenne Landfill 2196
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/8351083240221ddfd7c418c6d1bbdfb5-762cc373ef13afd74d2995f744d50bb9:getPixels


WY Eastern Laramie County SWDD 21136
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/eb6760d55d6bd925fcefbbc9e0c187ae-3d5746630e6585e3efe44fa91de8e541:getPixels


WY Sheridan Existing Landfill 11140
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/a6d1d4ed8cba2063f050bd8f2178e197-24f8bac457f1c30600cdf2a9d1ccc771:getPixels


In [ ]:
import random

def is_landfill(latitude, longitude):
    # Replace this function with your actual implementation to check if coordinates contain a landfill
    # For simplicity, I'm assuming no coordinates are landfills
    return False

def generate_random_coordinates(num_coordinates):
    coordinates = []
    while len(coordinates) < num_coordinates:
        # Generate random latitude and longitude in the contiguous United States
        # Bounding box for the contiguous U.S. coordinates
        latitude = random.uniform(24.396308, 49.384358)  # Lower and upper bounds for mainland U.S. latitude
        longitude = random.uniform(-125.000000, -66.934570)  # Lower and upper bounds for mainland U.S. longitude

        # Check if the coordinates contain a landfill
        if not is_landfill(latitude, longitude):
            coordinates.append((latitude, longitude))

    return coordinates

# Generate 1,000 random coordinates
random_coordinates = generate_random_coordinates(1000)

for i in range(30, 100):
    print(random_coordinates[i]);

(47.31019223260573, -95.83532677249892)
(25.908547301376764, -106.07198547793477)
(27.170026711513806, -95.79097784796096)
(44.11900357101409, -113.76702579719688)
(27.243957562493815, -69.08741507151255)
(42.62741776639125, -88.38010643216995)
(34.12664954646904, -81.08366357263395)
(25.704674232332895, -110.90108172659154)
(41.769282509201105, -82.08088910788382)
(25.67113251081903, -75.48154692302492)
(45.32215170441468, -73.66985368895762)
(41.11758954232271, -117.12238463815973)
(28.110087115541063, -102.00420916407897)
(38.84450354249047, -111.66817143341032)
(31.888377841215217, -89.6055168862994)
(31.392551741009104, -99.41384086425417)
(39.0113745402012, -122.15267547451248)
(28.19299650034616, -106.32844003713707)
(38.31690051487975, -104.3206105593696)
(27.29563292413014, -115.2298978098044)
(31.82862945327877, -120.32182129367322)
(28.642806614871468, -89.71260922765688)
(32.59912072177461, -103.2254195049205)
(41.55350379384564, -86.79431148669212)
(42.16206643602452, -105

In [ ]:
x = 1
for i in range(30, 100):
    print(x)
    x += 1
    print(random_coordinates[i])
    get_image_with_coordinates(random_coordinates[i][0], random_coordinates[i][1], 13)

1
(47.31019223260573, -95.83532677249892)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/f0eae853bd2960ba92bb94cb848f9d0b-9c2105707feb6833775dd9501d260a4c:getPixels


2
(25.908547301376764, -106.07198547793477)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/646fa1da8ce58ef373bb0294e57c3f27-6d19256474a7fb435ef82e63bf8a73ed:getPixels


3
(27.170026711513806, -95.79097784796096)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/55ed682b5c2f422eeca764b187268e22-f9774b1410ce92fa1b27fa418489faed:getPixels


4
(44.11900357101409, -113.76702579719688)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/3235246651285240c32df830fe0426b0-758c50feafcc56e370333f3f944ba3c5:getPixels


5
(27.243957562493815, -69.08741507151255)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/78b4bcbf92b49f40d8d8c8b9559ad096-e12adb04cd0d1e2e9b4c8e10f1526978:getPixels


...

66
(34.078645831418015, -78.6532696715755)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/fcd5915f2d0a92fb62f72b2ad7f2a697-cb8db08e02a8fa31bbf9cb6c4dd29217:getPixels


67
(32.90819535582301, -118.09544494293357)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/19e8e220e182b3685590edc297b4f07c-81416ec5efad8dd36913296b57b5d0ac:getPixels


68
(43.62348777146031, -95.2700794757715)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/de0d8818823ce7c942014bb2dda538c2-6030c731195eaefe80518e3beda61446:getPixels


69
(38.01510434354276, -119.03702056390688)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/b2eaf996398e46171e0a5eadbc687d04-bb9cc71ba65900397ba898e53a947da3:getPixels


70
(36.2910940864234, -84.2911606389506)
https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/53cfb0ccea1a14339bf084d71a06fe37-e3db514747f4b493cda8c11c7a9988ae:getPixels


In [ ]:
uploadedTwo = files.upload()

Saving Negative_Coordinates.xlsx to Negative_Coordinates (3).xlsx


In [ ]:
xlsx_file = io.BytesIO(uploadedTwo.get('Negative_Coordinates (3).xlsx'))
df = pd.read_excel(xlsx_file, usecols="B,C")
print(df)

       Latitude   Longitude
0     42.328674  -72.664658
1     42.341042  -71.217133
2     42.810356  -70.893875
3     41.638409  -70.941208
4     42.419331  -71.119720
...         ...         ...
1001  40.122500  -75.339800
1002  33.889400 -118.396700
1003  29.668900  -95.048400
1004  44.824700  -93.059600
1005  40.035300  -83.157800

[1006 rows x 2 columns]


In [ ]:
get_image_with_coordinates(48.8062624741948, -82.2943029497529, 13)

https://earthengine.googleapis.com/v1/projects/ee-my-amazing-username/thumbnails/1a39980526962d982bf0cf7bd5f084e9-b366e4f6d9a9e414ea88fae8c33878ee:getPixels


In [ ]:
i = 0
for index, row in df.iterrows():
    get_image_with_coordinates(row['Latitude'], row['Longitude'], i)
    i += 1